# Multi-product formulas (MPF)
{/* cspell:ignore Carrera, Multiproduct */}

Multi-product formulas (MPF) can be used to more accurately simulate the dynamics of a quantum system, at the cost of increased circuit executions. This is a post-processing technique that mitigates the error of expectation values for time-evolved states.

Classical computing is used to solve a system of linear equations that provides coefficients to a weighted combination of several circuit executions. Using this weighted combination can reduce the error associated with simulating time evolution, given a good selection of Trotter steps. The MPF tool will ingest a selection of data --- including the number of Trotter steps and the order of the Trotter approximation --- to prepare and solve (or approximate the solution of) the associated system of linear equations, which you can then use to post-process expectation value measurements of a time-evolved state.

## Install the MPF package

There are two ways to install the MPF package: via PyPI and building from source. It is recommended to install in a [virtual environment](https://docs.python.org/3.10/tutorial/venv.html) to ensure separation between package dependencies.

### Install from PyPI

The most straightforward way to install the `qiskit-addon-mpf` package is via PyPI.
```bash
pip install qiskit-addon-mpf
```

### Build from source
Users who wish to develop in the repository or who want to install it manually may do so by first cloning the repository:
```bash
git clone git@github.com:Qiskit/qiskit-addon-mpf.git
```

and install the package via `pip`. The repository also contains a number of optional dependencies that enable certain features.

Adjust the options to suit your needs.
```bash
pip install tox notebook -e '.[notebook-dependencies,dev]'
```

## Theoretical background

MPFs can reduce the Trotter approximation error associated with simulating the dynamics of quantum systems through a weighted combination of several circuit executions. This weighted sum is defined as:

$$ \mu(t) = \sum_j x_j\rho_j^{k_j}\left(\frac{t}{k_j}\right) + \text{some remaining Trotter error}, $$

where $x_j$ are the weighting coefficients, $\rho_j^{k_j}$ is the density matrix that corresponds to the pure state obtained by evolving the initial state via a product formula $S^{k_j}$ approximating the time-evolution operator using $k_j$ Trotter steps, and $j$ indexes each product formula used in the sum.

![Several circuits with a varying number of Trotter steps are used to compute the target observable](/images/guides/qiskit-addons/mpf-diagram.avif)

Generally, however, the goal of simulating quantum dynamics is to measure some observable $\mathcal{O}(t)$, which is a function of time. When using MPFs, multiple circuits -- each using $k_j$ Trotter steps -- are executed to obtain several measurements of the target observable $\mathcal{O}_{k_j}(t)$. The measurement of the target observable is then obtained by computing:

$$ \langle \mathcal{O}(t) \rangle = \sum_j x_j(t) \langle \mathcal{O}_{k_j}(t) \rangle. $$

In essence, you can reduce the overall Trotter error by approximating the time-evolution operator using several product formulas with a variable number of Trotter steps instead of a single product formula. You construct a circuit for each term in the weighted sum, which evolves the system according to each of the $k_j$ number of Trotter steps. Each circuit is then executed separately on a QPU to reconstruct the results in a post-processing step. The utility of this technique can be viewed from two perspectives:
1. For a fixed number of Trotter steps that you execute, you can obtain results with a Trotter error that is smaller in total.
2. For a number of Trotter steps that result in deep circuits, you can use MPF to find several shorter-depth circuits to run, which results in a similar Trotter approximation error.



## Determine MPF coefficients

The core functionality of the `qiskit-addon-mpf` package lies in determining the MPF coefficients $x_j(t)$ (which may be time-dependent). The process to obtain each $x_j(t)$ involves solving a system of linear equations $Ax=b$ where $x$ is the vector of coefficients to be determined, $A$ is a matrix that depends on each of the $k_j$ and the product formula used $S$ (as in, the approximation order and number of Trotter steps), and $b$ is a vector of constraints. This system of equations can be solved either exactly or with an approximate model that minimizes the 1-norm of the coefficients. Additionally, the choice for each $k_j$ is a heuristic process, but can be bounded by the following constraints:

1. The **largest** $k_j$ value is bounded by the highest depth of circuit that can be reliably executed
2. The **smallest** $k_j$ should satisfy $dt = t/k_j < 1$ since that is where the Trotter error is most well behaved
3. None of the coefficients $x_j$ should be close to $0$ since this would imply they don't contribute much to the MPF
4. Similarly, the coefficient associated with the largest $k_j$ value should not dominate, since this implies that you are using a single product formula
5. Lastly, the norm of the coefficients $x_j$ obtained should be small, as this indicates a well-conditioned MPF [1](#references)

## Next steps

<Admonition type="tip" title="Recommendations">
    - Read the page on [getting started with MPF](/guides/qiskit-addons-mpf-get-started).
</Admonition>

## References

1. A. Carrera Vazquez, D. J. Egger, D. Ochsner, and S. Wörner, ["Well-conditioned multi-product formulas for hardware-friendly Hamiltonian simulation"](https://quantum-journal.org/papers/q-2023-07-25-1067/), Quantum 7, 1067 (2023).
2. S. Zhuk, N. Robertson, and S. Bravyi, ["Trotter error bounds and dynamic multi-product formulas for Hamiltonian simulation"](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.6.033309), Phys. Rev. Research 6, 033309 (2024).
3. N. Robertson, et al. ["Tensor Network enhanced Dynamic Multiproduct Formulas"](https://arxiv.org/abs/2407.17405v2), arXiv:2407.17405v2 [quant-ph].